In [2]:
# from flask import Flask, request, jsonify
import os
from Audio2text.separate_audio import *
from Audio2text import diarize_model
from Denoiser.denoiser_run import denoise_audio
from S2T import *
from Speech_emo.emotion_predict import *
from Speech_emo import *
import time

In [5]:
file_save_path = "H://Learning Files/Project AI/Aidemo/aidemo/Audio_saving_file/Web_audio/y2mate.com - Dũng Ct Dảk Dảk Bủh Bủh Lmao Full HD.mp3"

In [6]:
start = time.time()
denoised_output_folder = "H://Learning Files/Project AI/Aidemo/aidemo/Audio_saving_file/Denoised/"
print("------Denosing audio------")
audio_denoised_link = denoise_audio(audio_link=file_save_path, output_folder=denoised_output_folder)
    
    # step 2: diaries audio using speaker diarization
print("------Diarizing audio------")
audio = whisperx.load_audio(audio_denoised_link) # load and embed audio for diarization
diarize_segments = get_DiarizeSegments(diarize_model = diarize_model,audio=audio, min_speakers = 2, max_speakers=2)
print(diarize_segments)
print('-----Segmenting audio-----')
threshold = 600.0
diarize_segments['duration'] = diarize_segments['end'] - diarize_segments['start']
diarize_segments = diarize_segments[diarize_segments['duration'] >= threshold]

segment_path = "H://Learning Files/Project AI/Aidemo/aidemo/Audio_saving_file/Segmented/"
diarize_segments = cut_and_save_segments(audio_link=audio_denoised_link, diarize_segments=diarize_segments, format_type = 'wav', save_path = segment_path)

# print(diarize_segments)
# bonus: combine audio
# print("------Combining audio------")
# audio_combine = combine_audio(diarize_segments, save_path= "H://Learning Files/Project AI/Aidemo/aidemo/Audio_saving_file/Combined/")
# step 3: speech emotion
print("------Predicting emotion------")
diarize_segments = predict_diarize_emo(diarize_segments)

# step 4: speech to text
print("------Transcribing audio------")
diarize_segments = text_diarize_save(diarize_segments)
end = time.time()
# step 5: return to client
print("-------Done--------, time: ", end-start)	


------Denosing audio------
Ran in 2.128126621246338s
------Diarizing audio------
Finished diarization in 2.5947067737579346s
                             segment label     speaker  start     end
0  [ 00:00:00.314 -->  00:00:03.556]     A  SPEAKER_00  310.0  3560.0
-----Segmenting audio-----
Saved y2mate_denoised_segment_1.wav
------Predicting emotion------
1/1 [==============================] - 0s 80ms/step
Finished predicting emotion in 0.3486781120300293s
------Transcribing audio------
-------Done--------, time:  40.97812271118164


In [7]:
diarize_segments

,segment,label,speaker,start,end,duration,link,emotion,text
0,[ 00:00:00.314 --> 00:00:03.556],A,SPEAKER_00,310.0,3560.0,3250.0,H://Learning Files/Project AI/Aidemo/aidemo/Au...,"[0.4664885699748993, 0.06364249438047409, 0.45...",{'text': 'đẳng đẳng bủ bủ lờ mau.'}


In [9]:
diarize_segments['text'][0]

{'text': 'a lô.'}

In [8]:
import torch

In [38]:
emotion_labels = ['Angry', 'Anxiety', 'Happy', 'Sad', 'Neutral']

In [9]:
# return 'A: xin chào (10% vui, 50% bình thường, 40% sợ)B: xin chào (10% vui, 50% bình thường, 40% sợ)', 200

string_output = ''
for i in range(len(diarize_segments)):
    id = diarize_segments.iloc[i]['speaker']
    text = diarize_segments.iloc[i]['text']['text']
    emotion = diarize_segments.iloc[i]['emotion']
    #  map to the emotion label
    
    temp = zip(emotion, emotion_labels)
    emotion = [f'{emo:.2f}% {label}' for emo, label in temp]
     
    string_output += id + ': ' + text + ' (' + ','.join(emotion) + ')'    

In [10]:
string_output

'SPEAKER_01: ờ. (0.00% Angry,0.00% Anxiety,100.00% Happy,0.00% Sad,0.00% Neutral)SPEAKER_01: tin này cho là minh viện tin nhắn. (99.91% Angry,0.00% Anxiety,0.09% Happy,0.00% Sad,0.00% Neutral)SPEAKER_01: quán ông xì ở đây là vùng vừa hỏi. (72.20% Angry,0.00% Anxiety,13.59% Happy,14.20% Sad,0.00% Neutral)SPEAKER_01: n. (19.39% Angry,0.00% Anxiety,80.56% Happy,0.05% Sad,0.00% Neutral)SPEAKER_01: mối bóng này ở những kỳ địa bát làng vùng cua có đường hạ của. (0.01% Angry,0.00% Anxiety,99.99% Happy,0.00% Sad,0.00% Neutral)SPEAKER_00: song quân á thì ăn. (0.05% Angry,0.00% Anxiety,99.95% Happy,0.00% Sad,0.00% Neutral)SPEAKER_01: của mình nó là cái gì chứ trời à phải sợ gì tự làm. (0.01% Angry,0.00% Anxiety,99.99% Happy,0.00% Sad,0.00% Neutral)SPEAKER_01: rồi thì mặt lạ gới. (7.23% Angry,0.00% Anxiety,0.39% Happy,92.38% Sad,0.01% Neutral)SPEAKER_01: tài chính nhì. (0.28% Angry,0.00% Anxiety,99.72% Happy,0.00% Sad,0.00% Neutral)SPEAKER_00: vâng vâng vâng. (0.00% Angry,0.00% Anxiety,100.00% Ha

In [3]:
import os

In [8]:
import os

# Thư mục con
subdir = "Denoised"

# Lấy đường dẫn đầy đủ của thư mục con
subdir_path = os.path.join(os.getcwd(), subdir)
print("Subdirectory Path:", subdir_path)


Subdirectory Path: h:\Learning Files\Project AI\Aidemo\aidemo\Denoise
